In [ ]:
from lib.llm.model import model_api_client, prompt_llm
from lib.prompting.prompts import create_operational_countries_prompt
import ast

# Main flow "24Finance" code
Describing the flow, in series, of the 24 Porto dashboard creation. 
Goal of the product: Show a dashboard with newsarticles that:
- presents per newsarticle whether it positively or negatively impacts your portfolio
- give the reason why
- give summary of the news
- quantification of the impact

### 1.1 Create and/or import the portfolio data table. With the following columns: [stockID, trading_market, industry, stock_position]
**TTwo types of position**
- 'long'
- 'short'

### 1.2 Add the operational countries per financial product
- Ask an LLM in what countries the company_name is involved in and fill in final 
- Final portfolio data tables should have columns: portfolio_table --> [stockID, company_name, trading_market, industry, type_of_product, OperationalCountries]

In [ ]:
client = model_api_client()
# prompt = "What are the countries in which ASML, the microchip company, operates? Return the answer as a python list of countries names only, no other text, such as ['el1', 'el2',...]."
role = "You are a financial assistent"
fillers={'stock_id': 'ASML'}

prompt = create_operational_countries_prompt(fillers)
countries = prompt_llm(client, prompt=prompt, role=role).choices[0].message.content
countries_list = ast.literal_eval(countries)


## 2 | News data processing & content matching
Goal: 
- Input all the news data
- Join them with the portfolio data table
- Add the news content


### 2.1 Import the news data tables
Import the following tables:
- cameo_country
- cameo_event
- news_data

### 2.2 Match the news data columns with the portfolio data 
Based on:
- Industry
- Involved countries

Output should be a table in which every row is a news article and the columns are : news_table --> [newsID, stockID, companyName, EventCode, EventRootCode, GoldsteinScale, AvgTone, Actor1_Geo_Country, Actor2_Geo_Country, ActionGeo_Country, numMentions, NewsUrl, OperationalCountries, TradeMarkets, NewsText, NewsSummary, BinaryImpact, ReasonCallout, RatingImpact]

Only the following columns are populated after this step:
- newsID
- stockID
- companyName
- EventCode
- EventRootCode
- GoldsteinScale
- AvgTone
- Actor1_Geo_Country
- Actor2_Geo_Country
- ActionGeo_Country
- numMentions
- NewsUrl
- OperationalCountries
- TradeMarkets

### 2.3 News scraping
Scrape the webpage denoted by NewsUrl to obtain the text of the news. 
Goal: populate the 'NewsText' column in the news_table table

## 3 | News x Stock comparison via LLM
Goal:
- Ask the LLM if the newsarticle (denoted by 'NewsText' column) has a postive or negative impact on the 'companyName' fin product
- Ask the LLM why the impact is as such
- (Ask the LLM to quantify the impact)
- Ask the LLM to summarize the news article (denoted in 'NewsText' column)

Python function: `def extract_fillers_from_matched_content(news_table_id:dict) -> dict:
    """
    {
        article_id:
        {
            stock_id:
            {
                "actor1_country":...,
                "actor2_country":...,
                "action_country":...,
                "article_text":...,
                ...
            }
        }
    }
    """`

Returns information for the folloiwing columns of the news_table table:
- NewsSummary
- BinaryImpact
- ReasonCallout
- RattingImpact


## 4 | Dashboarding 
Present the information in the 

In [ ]:
from lib.llm.model import model_api_client, prompt_llm
from lib.prompting.prompts import create_operational_countries_prompt
import ast

client = model_api_client()
prompt = "What are the countries in which ASML, the microchip company, operates? Return the answer as a python list of countries names only, no other text, such as ['el1', 'el2',...]."
role = "You are a financial assistent"
fillers={'stock_id': 'ASML'}

create_operational_countries_prompt(fillers)
countries = prompt_llm(client, prompt=prompt, role=role).choices[0].message.content
countries_list = ast.literal_eval(countries)


In [ ]:
# OUTPUT SAMPLE
# {
#   "id": "cmpl-8a9ba025b8a744e881636351a26e4642",
#   "object": "chat.completion",
#   "created": 1697721484,
#   "model": "zephyr-chat",
#   "choices": [
#       {
#           "index": 0,
#           "message": {
#               "role": "assistant",
#               "content": "There are typically 365 days in a year. However, in a leap year, which occurs every four years, there are 366 days. Leap years are used to account for the extra fraction of a day that is not included in a regular year."
#           },
#           "finish_reason": "stop"
#       }
#   ],
#   "usage": {
#       "prompt_tokens": 30,
#       "total_tokens": 90,
#       "completion_tokens": 60
#   }
# }